In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install -q pyspark

In [2]:

import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from scipy.sparse import dok_matrix
from pyspark.ml.linalg import Vectors

import math
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:

# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*


-rw-r--r-- 1 root root 247882 Mar 11 15:54 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [4]:

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *


In [5]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("YourAppName") \
    .config("spark.driver.memory", "32g") \
    .config("spark.executor.memory", "5940m") \
    .config("spark.executor.cores", "2") \
    .config("spark.driver.maxResultSize", "32g") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 15:57:09 INFO SparkEnv: Registering MapOutputTracker
24/03/11 15:57:09 INFO SparkEnv: Registering BlockManagerMaster
24/03/11 15:57:09 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/03/11 15:57:09 INFO SparkEnv: Registering OutputCommitCoordinator


In [6]:

# Put your bucket name below and make sure you can access it without an error
bucket_name = 'project_479_187' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name[0] == 'm':
        paths.append(full_path+b.name)


In [156]:
parquetFile = spark.read.parquet(*paths)
doc_title_pairs = parquetFile.select("id", "title").rdd

In [8]:
doc_title_pairs=doc_title_pairs.repartition(16)


In [159]:
doc_to_title = {id:(str(id),title) for id,title in doc_title_pairs.collectAsMap().items()}

In [9]:
class PorterStemmer:
    def __init__(self):
        self._vowels = {'a', 'e', 'i', 'o', 'u'}
        self._step1a_suffixes = [("sses", "ss"), ("ies", "i"), ("ss", "ss"), ("s", "")]
        self._step1b_suffixes = [("eed", "ee"), ("ed", ""), ("ing", "")]
        self._step2_suffixes = [
            ("ational", "ate"), ("tional", "tion"), ("enci", "ence"),
            ("anci", "ance"), ("izer", "ize"), ("bli", "ble"),
            ("alli", "al"), ("entli", "ent"), ("eli", "e"),
            ("ousli", "ous"), ("ization", "ize"), ("ation", "ate"),
            ("ator", "ate"), ("alism", "al"), ("iveness", "ive"),
            ("fulness", "ful"), ("ousness", "ous"), ("aliti", "al"),
            ("iviti", "ive"), ("biliti", "ble"), ("logi", "log")
        ]
        self._step3_suffixes = [
            ("icate", "ic"), ("ative", ""), ("alize", "al"),
            ("iciti", "ic"), ("ical", ""), ("ful", ""), ("ness", "")
        ]
        self._step4_suffixes = [
            ("al", ""), ("ance", ""), ("ence", ""), ("er", ""),
            ("ic", ""), ("able", ""), ("ible", ""), ("ant", ""),
            ("ement", ""), ("ment", ""), ("ent", ""),
            ("ou", ""), ("ism", ""), ("ate", ""), ("iti", ""),
            ("ous", ""), ("ive", ""), ("ize", "")
        ]
        self._step5a_suffixes = [("e", ""), ("e", "")]
        self._step5b_suffixes = [("ll", "l")]

    def _is_consonant(self, word, index):
        if index < 0 or index >= len(word):  # Ensure index is within bounds
              return False
        else:
            if word[index] in self._vowels:
                  return False
            if word[index] == 'y':
                  return index == 0 or not self._is_consonant(word, index - 1)
        return True


    def _measure(self, word):
        """
        Calculate the measure of a word, the number of consonant sequences.
        """
        count = 0
        i = 0
        while i < len(word):
            while i < len(word) and not self._is_consonant(word, i):
                i += 1
            if i < len(word):
                count += 1
                while i < len(word) and self._is_consonant(word, i):
                    i += 1
        return count
    def _contains_vowel(self, word):
        """
        Check if a word contains a vowel.
        """
        for char in word:
            if char in self._vowels:
                return True
        return False

    def _replace_suffix(self, word, old_suffix, new_suffix):
        """
        Replace the old suffix of a word with a new suffix.
        """
        if word.endswith(old_suffix):
            return word[:-len(old_suffix)] + new_suffix
        return word

    def _apply_rule(self, word, suffixes):
        """
        Apply a list of suffix replacement rules to a word.
        """
        for old_suffix, new_suffix in suffixes:
            if word.endswith(old_suffix):
                stemmed_word = self._replace_suffix(word, old_suffix, new_suffix)
                if self._measure(stemmed_word) > 0:
                    return stemmed_word
                else:
                    return word
        return word

    def stem(self, word):
        """
        Apply the Porter stemming algorithm to a word.
        """
        if len(word) < 3:
            return word

        word = self._apply_rule(word, self._step1a_suffixes)

        if word.endswith("y") and self._contains_vowel(word[:-1]):
            word = word[:-1] + "i"

        word = self._apply_rule(word, self._step1b_suffixes)

        word = self._apply_rule(word, self._step2_suffixes)

        word = self._apply_rule(word, self._step3_suffixes)

        word = self._apply_rule(word, self._step4_suffixes)

        if word.endswith("e"):
            stemmed_word = self._replace_suffix(word, "e", "")
            if self._measure(stemmed_word) > 1:
                word = stemmed_word
            elif self._measure(stemmed_word) == 1 and not self._is_consonant(stemmed_word, -2):
                word = stemmed_word

        if self._measure(word) > 1 and word.endswith("ll"):
            word = word[:-1]

        return word


In [45]:
# -*- coding: utf-8 -*-
"""Untitled13.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1-96zO3pwOgfY2ST2rhlwseOZ8R8V2qsR
"""

import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
from time import time
from pathlib import Path
import pickle
from google.cloud import storage
from collections import defaultdict
from contextlib import closing
import math

PROJECT_ID = 'YOUR-PROJECT-ID-HERE'
def get_bucket(bucket_name):
    return storage.Client(PROJECT_ID).bucket(bucket_name)

def _open(path, mode, bucket=None):
    if bucket is None:
        return open(path, mode)
    return bucket.blob(path).open(mode)

# Let's start with a small block size of 30 bytes just to test things out.
BLOCK_SIZE = 1999998

class MultiFileWriter:
    """ Sequential binary writer to multiple files of up to BLOCK_SIZE each. """
    def __init__(self, base_dir, name, bucket_name=None):
        self._base_dir = Path(base_dir)
        self._name = name
        self._bucket = None if bucket_name is None else get_bucket(bucket_name)
        self._file_gen = (_open(str(self._base_dir / f'{name}_{i:03}.bin'),
                                'wb', self._bucket)
                          for i in itertools.count())
        self._f = next(self._file_gen)

    def write(self, b):
        locs = []
        while len(b) > 0:
            pos = self._f.tell()
            remaining = BLOCK_SIZE - pos
            # if the current file is full, close and open a new one.
            if remaining == 0:
                self._f.close()
                self._f = next(self._file_gen)
                pos, remaining = 0, BLOCK_SIZE
            self._f.write(b[:remaining])
            name = self._f.name if hasattr(self._f, 'name') else self._f._blob.name
            locs.append((name, pos))
            b = b[remaining:]
        return locs

    def close(self):
        self._f.close()

class MultiFileReader:
    """ Sequential binary reader of multiple files of up to BLOCK_SIZE each. """
    def __init__(self, base_dir, bucket_name=None):
        self._base_dir = Path(base_dir)
        self._bucket = None if bucket_name is None else get_bucket(bucket_name)
        self._open_files = {}

    def read(self, locs, n_bytes):
        b = []
        for f_name, offset in locs:
            f_name = str(self._base_dir / f_name)
            if f_name not in self._open_files:
                self._open_files[f_name] = _open(f_name, 'rb', self._bucket)
            f = self._open_files[f_name]
            f.seek(offset)
            n_read = min(n_bytes, BLOCK_SIZE - offset)
            b.append(f.read(n_read))
            n_bytes -= n_read
        return b''.join(b)

    def close(self):
        for f in self._open_files.values():
            f.close()

    def __exit__(self, exc_type, exc_value, traceback):
        self.close()
        return False

TUPLE_SIZE = 6       # We're going to pack the doc_id and tf values in this
                     # many bytes.
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer


class InvertedIndex:
    def __init__(self, docs={}):
        """ Initializes the inverted index and add documents to it (if provided).
        Parameters:
        -----------
          docs: dict mapping doc_id to list of tokens
        """
        # stores document frequency per term
        self.df = Counter()
        # stores total frequency per term
        self.term_total = Counter()
        # stores posting list per term while building the index (internally),
        # otherwise too big to store in memory.
        self._posting_list = defaultdict(list)
        # mapping a term to posting file locations, which is a list of
        # (file_name, offset) pairs. Since posting lists are big we are going to
        # write them to disk and just save their location in this list. We are
        # using the MultiFileWriter helper class to write fixed-size files and store
        # for each term/posting list its list of locations. The offset represents
        # the number of bytes from the beginning of the file where the posting list
        # starts.
        self.posting_locs = defaultdict(list)
        self.n = len(docs)
        self.docs=docs.keys()
        self.nf={id:len(text) for id, text in docs.items()}

        for doc_id, tokens in docs.items():
            self.add_doc(doc_id, tokens)

    def calc_tf_idf(self, token, tf):
        df = self.df[token]
        n = self.n
        idf = math.log(n/df)
        return tf * idf

    def add_doc(self, doc_id, tokens):
        """ Adds a document to the index with a given `doc_id` and tokens. It counts
            the tf of tokens, then update the index (in memory, no storage
            side-effects).
        """
        w2cnt = Counter(tokens)
        self.term_total.update(w2cnt)
        for w, cnt in w2cnt.items():
            self.df[w] = self.df.get(w, 0) + 1
            tf_idf = self.calc_tf_idf(w, cnt)
            self._posting_list[w].append((doc_id, tf_idf))


    def write_index(self, base_dir, name, bucket_name=None):
        """ Write the in-memory index to disk. Results in the file:
            (1) `name`.pkl containing the global term stats (e.g. df).
        """
        #### GLOBAL DICTIONARIES ####
        self._write_globals(base_dir, name, bucket_name)

    def _write_globals(self, base_dir, name, bucket_name):
        path = str(Path(base_dir) / f'{name}.pkl')
        bucket = None if bucket_name is None else get_bucket(bucket_name)
        with _open(path, 'wb', bucket) as f:
            pickle.dump(self, f)

    def __getstate__(self):
        """ Modify how the object is pickled by removing the internal posting lists
            from the object's state dictionary.
        """
        state = self.__dict__.copy()
        del state['_posting_list']
        return state

    def posting_lists_iter(self, base_dir, bucket_name=None):
        """ A generator that reads one posting list from disk and yields
            a (word:str, [(doc_id:int, tf:int), ...]) tuple.
        """
        with closing(MultiFileReader(base_dir, bucket_name)) as reader:
            for w, locs in self.posting_locs.items():
                b = reader.read(locs, self.df[w] * TUPLE_SIZE)
                posting_list = []
                for i in range(self.df[w]):
                    doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
                    tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
                    posting_list.append((doc_id, tf))
                yield w, posting_list

    def read_a_posting_list(self, base_dir, w, bucket_name=None):
        posting_list = []
        if not w in self.posting_locs:
            return posting_list
        with closing(MultiFileReader(base_dir, bucket_name)) as reader:
            locs = self.posting_locs[w]
            b = reader.read(locs, self.df[w] * TUPLE_SIZE)
            for i in range(self.df[w]):
                doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
                tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
                posting_list.append((doc_id, tf))
        return posting_list

    @staticmethod
    def write_a_posting_list(b_w_pl, base_dir, bucket_name=None):
        posting_locs = defaultdict(list)
        bucket_id, list_w_pl = b_w_pl

        with closing(MultiFileWriter(base_dir, bucket_id, bucket_name)) as writer:
            for w, pl in list_w_pl:
                # convert to bytes
                b = b''.join([(doc_id << 16 | (tf & TF_MASK)).to_bytes(TUPLE_SIZE, 'big')
                              for doc_id, tf in pl])
                # write to file(s)
                locs = writer.write(b)
                # save file locations to index
                posting_locs[w].extend(locs)
            path = str(Path(base_dir) / f'{bucket_id}_posting_locs.pickle')
            bucket = None if bucket_name is None else get_bucket(bucket_name)
            with _open(path, 'wb', bucket) as f:
                pickle.dump(posting_locs, f)
        return bucket_id


    @staticmethod
    def read_index(base_dir, name, bucket_name=None):
        path = str(Path(base_dir) / f'{name}.pkl')
        bucket = None if bucket_name is None else get_bucket(bucket_name)
        with _open(path, 'rb', bucket) as f:
            return pickle.load(f)

In [26]:
import struct

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){0,24}""", re.UNICODE)
def tokenize(text):
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token.group()) for token in RE_WORD.finditer(text.lower())]
    return [token for token in tokens if token not in all_stopwords]

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
    ''' Count the frequency of each word in text (tf) that is not included in
    all_stopwords and return entries that will go into our posting lists.
    Parameters:
    -----------
    text: str
      Text of one document
    id: int
      Document id
    Returns:
    --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
    '''
    stemmer = PorterStemmer()
    tokens = []
    alt_tokens = []
    for token in RE_WORD.finditer(text.lower()):
        tok = token.group()
        if tok not in all_stopwords:
            st_tok = stemmer.stem(tok)
            tokens.append(st_tok)
            alt_tokens.append(tok)


    count = Counter(tokens)
    if len(count)==0:
        count = Counter(alt_tokens)
    res = []
    for tup in count.items():
        res.append((tup[0],(id,tup[1])))
    return res

def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
    Returns:
    --------
    list of tuples
      A sorted posting list.
    '''
    # YOUR CODE HERE
    sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
    return sorted_pl

def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each element is a (token, df) pair.
    '''
    # YOUR CODE HERE
    return postings.map(lambda x: (x[0], len(x[1])))

from math import log

def calculate_tf_idf(postings, total_docs):
    ''' Calculates the TF-IDF for each token.
    Parameters:
    -----------
      postings: RDD
        An RDD where each element is a (token, postings_list) pair, and
        postings_list is a list of (document_id, term_frequency) tuples for that token.
      total_docs: int
        The total number of documents in the corpus.
    Returns:
    --------
      RDD
        An RDD where each element is a (token, [(document_id, tf_idf), ...]) pair.
    '''
    # Calculate IDF for each token
    # Note: log base is e by default, which is common in TF-IDF calculations
    def idf(df):
        return log(total_docs / df)
    
    # Calculate TF-IDF for each (document_id, term_frequency) tuple in the postings list
    def tf_idf(postings_list):
        df = len(postings_list)
        return [(doc_id, tf * idf(df)) for doc_id, tf in postings_list]
    
    return postings.map(lambda x: (x[0], tf_idf(x[1])))

def partition_postings_and_write(postings):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
    '''
    # YOUR CODE HERE
    buckets = postings.map(lambda x: (token2bucket_id(x[0]),(x[0],x[1]))).groupByKey().mapValues(list)
    res = buckets.map(lambda x: InvertedIndex().write_a_posting_list(x,"text/", bucket_name))
    return res

def partition_postings_and_write_title(postings):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
    '''
    # YOUR CODE HERE
    buckets = postings.map(lambda x: (token2bucket_id(x[0]),(x[0],x[1]))).groupByKey().mapValues(list)
    res = buckets.map(lambda x: InvertedIndex().write_a_posting_list(x, "title/",bucket_name))
    return res

In [27]:
def get_document_vectors(inverted_index, vocab):
    """
    This function retrieves document vectors from an inverted index with TF-IDF values.

    Args:
      inverted_index: A dictionary where keys are unique terms and values are postings lists.
          Each postings list is a list of tuples (document_id, tfidf_value).

    Returns:
      A dictionary where keys are document IDs and values are lists representing
          sparse document vectors (containing only non-zero TF-IDF values).
    """
    document_vectors = {}
    for term, postings_list in inverted_index:
        for document_id, tfidf_value in postings_list:
          # Leverage document_id as the index for the sparse vector
            sparse_vec_indices = [vocab[term]]
            sparse_vec_values = [tfidf_value]
            # Update or initialize sparse vector for the document
            if document_id in document_vectors:
                  document_vectors[document_id].extend(zip(sparse_vec_indices, sparse_vec_values))
            else:
                  document_vectors[document_id] = list(zip(sparse_vec_indices, sparse_vec_values))
    # Convert the lists of indices and values to sparse vectors
    for doc, lst in document_vectors.items():
        document_vectors[doc]=sorted(lst, key=lambda x: x[0])
    document_vectors = {doc_id: Vectors.sparse(len(vocab), [x[0] for x in lst], [x[1] for x in lst]) for doc_id, lst in document_vectors.items()}
    return document_vectors


In [28]:
word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
w2df = calculate_tf_idf(postings, parquetFile.count())
vocab = w2df.flatMap(lambda x: [x[0]]).distinct().collect()
vocab_dct_title = {term:x for x,term in enumerate(vocab)}



vocab done


In [29]:
pst = w2df.collect()

vec done


In [41]:
w2dfdict=w2df.collectAsMap()

In [165]:
pickle_file_path = 'w2dfdict.pkl'

# Save the dictionary as a pickle file in the bucket
with open('w2dfdict.pkl', 'wb') as f:
    pickle.dump(doc_to_title, f)

# Create a blob object for the pickle file
blob = bucket.blob(pickle_file_path)

# Upload the pickle file to the bucket
blob.upload_from_filename('w2dfdict.pkl')

In [47]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [49]:
from inverted_index_gcp import InvertedIndex

In [168]:
bucket_name = 'project_479_187' 
doc_to_title_file_path = 'doc_to_title.pkl'
bucket = client.get_bucket(bucket_name)
blob_doc_to_title = bucket.blob(doc_to_title_file_path)

# Load the pickle file into a dictionary
doc_to_title1 = pickle.loads(blob_doc_to_title.download_as_bytes())

pst_file_path = 'pst.pkl'
blob_pst = bucket.blob(pst_file_path)

# Load the pickle file into a dictionary
pst = pickle.loads(blob_pst.download_as_bytes())

w2dfdict_file_path = 'w2dfdict.pkl'
blob_w2dfdict = bucket.blob(w2dfdict_file_path)

# Load the pickle file into a dictionary
w2dfdict = pickle.loads(blob_w2dfdict.download_as_bytes())

import numpy as np
import time
def search(query):
    ''' Returns up to a 100 of your best search results for the query. This is
        the place to put forward your best search engine, and you are free to
        implement the retrieval whoever you'd like within the bound of the
        project requirements (efficiency, quality, etc.). That means it is up to
        you to decide on whether to use stemming, remove stopwords, use
        PageRank, query expansion, etc.

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each
        element is a tuple (wiki_id, title).
    '''
    # BEGIN SOLUTION
#     def load_item_from_pickle(filename):
#         with open(filename, 'rb') as file:
#           return pickle.load(file)
#     title_tf_idf=load_item_from_pickle('title_tf_idf.pkl')
#     text_tf_idf=load_item_from_pickle('text_tf_idf.pkl')
#     pagerank=load_item_from_pickle('pagerank.pkl')
#     allstopwords=load_item_from_pickle('allstopwords.pkl')
    def create_sparse_vector_from_counter(query_counter, vocab):

        """

        This function creates a sparse vector from a query represented as a counter.

        Args:
          query_counter: A Counter object representing the query, where keys are tokens and values are counts.
          vocab: A list of unique terms.

        Returns:
          A sparse vector representing the query, with non-zero counts at indices corresponding to tokens in the vocab.
        """
        # Initialize empty lists for indices and values
        sparse_vec_indices = []
        sparse_vec_values = []
        val_idx = {}
        # Iterate over tokens in the query counter
        for token, count in query_counter.items():
          # Get the index of the token in the vocab list
          if token in vocab:
            term_index = vocab[token]
            # Append the index and count to the sparse vector
#             sparse_vec_indices.append(term_index)
#             sparse_vec_values.append(count)
            val_idx[term_index]=count
        # Create a sparse vector from the indices and values
        val_idx = dict(sorted(val_idx.items(), key=lambda x: x[0]))
        sparse_vector = Vectors.sparse(len(vocab), list(val_idx.keys()), list(val_idx.values()))

        return sparse_vector


    np.seterr(divide='ignore', invalid='ignore')
    stemmer=PorterStemmer()
    RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
    tokens = [stemmer.stem(token.group()) if len(token.group()) > 1 else token.group() for token in RE_WORD.finditer(query.lower())]
    doc_ids = [doc_id for token in tokens for doc_id, _ in w2dfdict[token] if token in w2dfdict]
    count = Counter([token for token in tokens])
    q_vec_title = create_sparse_vector_from_counter(count,vocab_dct_title)
    # Calculate the cosine similarity between the query vector and the document vectors


    stemmer=PorterStemmer()
    RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
    tokens = [stemmer.stem(token.group()) if len(token.group()) > 1 else token.group() for token in RE_WORD.finditer(query.lower())]
    filtered_vocab = {k: v for k,v in vocab_dct_title.items() if k in tokens}
    pst_filtered = [(term, postings_list) for term, postings_list in pst if term in filtered_vocab]
    document_vectors = get_document_vectors(pst_filtered, vocab_dct_title)
    # Calculate the cosine similarity between the query vector and the document vectors
    def cosine_similarity(vec, column_vector):
        """
        Calculate the cosine similarity between two vectors.
        """
        dot_product = vec.dot(column_vector)
        norm_vec = vec.norm(2)
        norm_column_vector = column_vector.norm(2)
        similarity = dot_product / (norm_vec * norm_column_vector)
        return similarity

    res = []
    for doc_id, vec in document_vectors.items():
        sim_title = cosine_similarity(q_vec_title, vec)
        res.append((doc_id, sim_title))
    docs=[x[0] for x in sorted(res, key=lambda x: x[1], reverse=True)[:30]]


    # Download the pickle file from the bucket to a local file
#     bucket = client.get_bucket(bucket_name)
#     blob = bucket.blob(pickle_file_path)
#     blob.download_to_filename('doc_to_title.pkl')
    values = [doc_to_title1[key] for key in docs if key in doc_to_title1]
    return values

In [169]:
search("mona lisa")

[('70889', 'Mona Lisa'),
 ('168571', 'Mona Lisa (1986 film)'),
 ('349909', 'Mona Lisa Overdrive'),
 ('883489', 'Mona Lisa Smile'),
 ('1197449', '1000 Mona Lisas'),
 ('2683924', 'Mona Lisa (opera)'),
 ('3378775', 'Mona Lisa (disambiguation)'),
 ('4331031', 'Mona Lisa Overdrive (album)'),
 ('6185069', 'Isleworth Mona Lisa'),
 ('6906354', 'Mona Lisa (singer)'),
 ('9587382', 'Mona Lisas and Mad Hatters'),
 ('10393560', 'Mona Lisas and Mad Hatters (Part Two)'),
 ('12425249', 'The Theft of the Mona Lisa'),
 ('13016965', 'I, Mona Lisa'),
 ('13893541', 'Mona Lisa (Nat King Cole song)'),
 ('14623843', 'Mona Lisa (crater)'),
 ('15239160', 'Speculations about Mona Lisa'),
 ('16898689', 'Mona Lisa Yuchengco'),
 ('16927651', 'Mona Lisa (actress)'),
 ('20210450', 'Playing Mona Lisa'),
 ('23909451', 'Monday Mona Lisa Club'),
 ('26381501', 'Mona Lisa & Others'),
 ('29130029', 'Mona Lisa Lost Her Smile'),
 ('30512525', 'The Ballad of Mona Lisa'),
 ('32363829', 'Mona Lisa (EP)'),
 ('36796972', 'Mona Lis